In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['SG', 'SGs']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('stress granules', 448),
 ('sleeve gastrectomy', 135),
 ('salivary gland', 115),
 ('stress granule', 107),
 ('substantia gelatinosa', 67),
 ('spiral ganglion', 36),
 ('sun ginseng', 30),
 ('sebaceous gland', 28),
 ('secretory granule', 24),
 ('stellate ganglion', 17),
 ('satratoxin g', 16),
 ('subgenomic', 16),
 ('stratum granulosum', 12),
 ('stellate ganglia', 12),
 ('sarcoglycan', 11),
 ('salmonella gallinarum', 10),
 ('steviol glycosides', 10),
 ('sybr green i', 8),
 ('sophoraflavanone g', 7),
 ('savitzky golay', 6),
 ('standard gamble', 6),
 ('spring germination', 6),
 ('serious games', 5),
 ('suicide gene', 5),
 ('structural genomic', 5),
 ('sulfated galactans', 5),
 ('sytox green', 5),
 ('standard germination', 5),
 ('schistosome generated', 5),
 ('steryl glucosides', 5),
 ('sympathetic ganglia', 5),
 ('sanguinarine', 5),
 ('sinus gland', 5),
 ('sorafenib β d glucuronide', 5),
 ('sterylglucosides', 5),
 ('sweat glands', 5),
 ('slow gamma', 4),
 ('skin grafting', 4),
 ('sarcodio

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-09 02:59:55] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'salivary gland': 'MESH:D012469',
  'salmonella gallinarum': 'Taxonomy:594',
  'sangiovese': 'ungrounded',
  'sanguinarine': 'CHEBI:CHEBI:17183',
  'sarcodiotheca gaudichaudii': 'ungrounded',
  'sarcoglycan': 'FPLX:Sarcoglycan_complex',
  'sarcoglycans': 'FPLX:Sarcoglycan_complex',
  'sargachromanol g': 'MESH:C577249',
  'satratoxin g': 'MESH:C028268',
  'savitzky golay': 'ungrounded',
  'schistosome generated': 'ungrounded',
  'sebaceous gland': 'MESH:D012627',
  'secretory granule': 'GO:GO:0030141',
  'serglycin': 'HGNC:9361',
  'serious games': 'ungrounded',
  'serovar gallinarum': 'ungrounded',
  'sesaminol glucosides': 'CHEBI:CHEBI:24278',
  'sesbania gum': 'ungrounded',
  'shenzhuguanxin granule': 'ungrounded',
  'siegesbeckia glabrescens': 'ungrounded',
  'sinus gland': 'ungrounded',
  'sitagliptin': 'CHEBI:CHEBI:40237',
  'skin grafting': 'MESH:D016038',
  'sleeve gastrectomy': 'MESH:D005743',
  'slow gamma': 'ungrounded',
  'sophoraflavanone g': 'CHEBI:CHEBI:50209',
  'soraf

In [26]:
grounding_map, names, pos_labels = [{'salivary gland': 'MESH:D012469',
  'salmonella gallinarum': 'TAXONOMY:594',
  'sangiovese': 'ungrounded',
  'sanguinarine': 'CHEBI:CHEBI:17183',
  'sarcodiotheca gaudichaudii': 'ungrounded',
  'sarcoglycan': 'FPLX:Sarcoglycan_complex',
  'sarcoglycans': 'FPLX:Sarcoglycan_complex',
  'sargachromanol g': 'MESH:C577249',
  'satratoxin g': 'MESH:C028268',
  'savitzky golay': 'ungrounded',
  'schistosome generated': 'ungrounded',
  'sebaceous gland': 'MESH:D012627',
  'secretory granule': 'GO:GO:0030141',
  'serglycin': 'HGNC:9361',
  'serious games': 'ungrounded',
  'serovar gallinarum': 'ungrounded',
  'sesaminol glucosides': 'CHEBI:CHEBI:24278',
  'sesbania gum': 'ungrounded',
  'shenzhuguanxin granule': 'ungrounded',
  'siegesbeckia glabrescens': 'ungrounded',
  'sinus gland': 'ungrounded',
  'sitagliptin': 'CHEBI:CHEBI:40237',
  'skin grafting': 'MESH:D016038',
  'sleeve gastrectomy': 'MESH:D005743',
  'slow gamma': 'ungrounded',
  'sophoraflavanone g': 'CHEBI:CHEBI:50209',
  'sorafenib β d glucuronide': 'ungrounded',
  'spiral ganglion': 'MESH:D013136',
  'spring germination': 'ungrounded',
  'stacked graphene': 'CHEBI:CHEBI:36973',
  'standard gamble': 'ungrounded',
  'standard germination': 'ungrounded',
  'starch grains': 'ungrounded',
  'stellate ganglia': 'MESH:D013233',
  'stellate ganglion': 'MESH:D013233',
  'steryl glucosides': 'ungrounded',
  'sterylglucosides': 'ungrounded',
  'steviol glycosides': 'CHEBI:CHEBI:145027',
  'stratum granulosum': 'NCIT:C33627',
  'stress granule': 'GO:GO:0010494',
  'stress granules': 'GO:GO:0010494',
  'striae gravidarum': 'ungrounded',
  'structural genetic': 'ungrounded',
  'structural genomic': 'ungrounded',
  'subesophageal ganglion': 'MESH:D045888',
  'subgenomic': 'ungrounded',
  'substantia gelatinosa': 'MESH:D013376',
  'subtotal gastrectomy': 'MESH:D005743',
  'suicide gene': 'MESH:D041722',
  'sulfated galactans': 'CHEBI:CHEBI:37165',
  'sun ginseng': 'MESH:D005894',
  'sweat gland': 'MESH:D013545',
  'sweat glands': 'MESH:D013545',
  'switchgrass': 'ungrounded',
  'sybr green i': 'CHEBI:CHEBI:51461',
  'sympathetic ganglia': 'MESH:D005728',
  'sytox green': 'MESH:C402795'},
 {'MESH:D012469': 'Salivary Glands',
  'TAXONOMY:594': 'Salmonella enterica subsp. enterica serovar Gallinarum',
  'CHEBI:CHEBI:17183': 'sanguinarine',
  'FPLX:Sarcoglycan_complex': 'Sarcoglycan_complex',
  'MESH:C577249': 'sargachromanol G',
  'MESH:C028268': 'satratoxin G',
  'MESH:D012627': 'Sebaceous Glands',
  'GO:GO:0030141': 'secretory granule',
  'HGNC:9361': 'SRGN',
  'CHEBI:CHEBI:24278': 'glucoside',
  'CHEBI:CHEBI:40237': 'sitagliptin',
  'MESH:D016038': 'Skin Transplantation',
  'MESH:D005743': 'Gastrectomy',
  'CHEBI:CHEBI:50209': 'sophoraflavanone G',
  'MESH:D013136': 'Spiral Ganglion',
  'CHEBI:CHEBI:36973': 'graphene',
  'MESH:D013233': 'Stellate Ganglion',
  'CHEBI:CHEBI:145027': 'steviol glycoside',
  'NCIT:C33627': 'Stratum Granulosum',
  'GO:GO:0010494': 'cytoplasmic stress granule',
  'MESH:D045888': 'Ganglion Cysts',
  'MESH:D013376': 'Substantia Gelatinosa',
  'MESH:D041722': 'Genes, Transgenic, Suicide',
  'CHEBI:CHEBI:37165': 'galactan',
  'MESH:D005894': 'Panax',
  'MESH:D013545': 'Sweat Glands',
  'CHEBI:CHEBI:51461': 'SYBR Green I',
  'MESH:D005728': 'Ganglia, Sympathetic',
  'MESH:C402795': 'SYTOX Green'},
 ['FPLX:Sarcoglycan_complex',
  'GO:GO:0010494',
  'GO:GO:0030141',
  'MESH:C028268',
  'MESH:D005743',
  'MESH:D005894',
  'MESH:D012469',
  'MESH:D012627',
  'MESH:D013136',
  'MESH:D013376',
  'MESH:D013233',
  'MESH:D013545',
  'TAXONOMY:594',
  'NCIT:C33627']]

In [10]:
excluded_longforms = []

In [27]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [12]:
additional_entities = {}

In [13]:
unambiguous_agent_texts = {}

In [28]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [15]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [16]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [17]:
intersection1

[('HGNC:17981', 'HGNC:17981', 29)]

In [18]:
intersection2

[('ungrounded', 'HGNC:17981', 0)]

In [20]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['RTCA', 'RTCD1', 'RPC', 'RTC1', 'RTC'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [ ]:
names.update(additional_entitie)

In [29]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-09 03:47:53] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-09 03:48:34] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9349396104362497 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [21]:
classifier.stats

{'label_distribution': {'ungrounded': 59,
  'MESH:D013545': 10,
  'GO:GO:0030141': 21,
  'MESH:D045888': 2,
  'MESH:D012627': 24,
  'MESH:D013376': 48,
  'MESH:D013233': 20,
  'MESH:D016038': 2,
  'MESH:D012469': 73,
  'FPLX:Sarcoglycan_complex': 15,
  'MESH:D005743': 90,
  'MESH:D013136': 28,
  'HGNC:9361': 3,
  'MESH:C028268': 11,
  'GO:GO:0010494': 426,
  'CHEBI:CHEBI:51461': 6,
  'Taxonomy:594': 8,
  'CHEBI:CHEBI:24278': 3,
  'MESH:D005894': 10,
  'NCIT:C33627': 8,
  'MESH:C402795': 3,
  'MESH:D005728': 2,
  'MESH:D041722': 2,
  'MESH:C577249': 3,
  'CHEBI:CHEBI:17183': 4,
  'CHEBI:CHEBI:37165': 5,
  'CHEBI:CHEBI:145027': 9,
  'CHEBI:CHEBI:50209': 5,
  'CHEBI:CHEBI:36973': 1,
  'CHEBI:CHEBI:40237': 2},
 'f1': {'mean': 0.936376, 'std': 0.007423},
 'precision': {'mean': 0.928634, 'std': 0.00314},
 'recall': {'mean': 0.957904, 'std': 0.009549},
 'MESH:D013136': {'f1': {'mean': 0.95, 'std': 0.1},
  'pr': {'mean': 0.92, 'std': 0.16},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D041722': 

In [30]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [32]:
disamb.dump(model_name, results_path)

In [31]:
print(disamb.info())

Disambiguation model for SG, and SGs

Produces the disambiguations:
	Ganglia, Sympathetic	MESH:D005728
	Ganglion Cysts	MESH:D045888
	Gastrectomy*	MESH:D005743
	Genes, Transgenic, Suicide	MESH:D041722
	Panax*	MESH:D005894
	SRGN	HGNC:9361
	SYBR Green I	CHEBI:CHEBI:51461
	SYTOX Green	MESH:C402795
	Salivary Glands*	MESH:D012469
	Salmonella enterica subsp. enterica serovar Gallinarum*	TAXONOMY:594
	Sarcoglycan_complex*	FPLX:Sarcoglycan_complex
	Sebaceous Glands*	MESH:D012627
	Skin Transplantation	MESH:D016038
	Spiral Ganglion*	MESH:D013136
	Stellate Ganglion*	MESH:D013233
	Stratum Granulosum*	NCIT:C33627
	Substantia Gelatinosa*	MESH:D013376
	Sweat Glands*	MESH:D013545
	cytoplasmic stress granule*	GO:GO:0010494
	galactan	CHEBI:CHEBI:37165
	glucoside	CHEBI:CHEBI:24278
	graphene	CHEBI:CHEBI:36973
	sanguinarine	CHEBI:CHEBI:17183
	sargachromanol G	MESH:C577249
	satratoxin G*	MESH:C028268
	secretory granule*	GO:GO:0030141
	sitagliptin	CHEBI:CHEBI:40237
	sophoraflavanone G	CHEBI:CHEBI:50209
	stevi

In [29]:
model_to_s3(disamb)